## Work in Progress -- Use the AIW dataset to benchmark o1-preview against r1

In [1]:
!pip install -qU openai datasets tqdm pandas

In [2]:
import getpass
openai_api_key = getpass.getpass("OpenAI API key: ")


In [3]:
from datasets import load_dataset

# Load the "Alice in Wonderland" dataset from Hugging Face
dataset = load_dataset("marianna13/aiw-prompts")
data = dataset['test'] if 'test' in dataset else dataset['train']

/Users/Angela/Desktop/genai-demos/benchmarking_o1_vs_r1/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 274/274 [00:00<00:00, 37538.44 examples/s]


In [7]:
print(data[:5])  # Inspect the first 5 rows of 'data'


{'id': [1, 2, 3, 4, 5], 'text': ["Alice has 4 brothers and she also has 1 sister. How many sisters does Alice's brother have? To answer the question, DO NOT OUTPUT ANY TEXT EXCEPT following format that contains final answer: ### Answer:", "Alice has 4 sisters and she also has 1 brother. How many sisters does Alice's brother have? To answer the question, DO NOT OUTPUT ANY TEXT EXCEPT following format that contains final answer: ### Answer:", "Alice has four brothers and she also has one sister. How many sisters does Alice's brother have? To answer the question, DO NOT OUTPUT ANY TEXT EXCEPT following format that contains final answer: ### Answer:", "Alice has four sisters and she also has one brother. How many sisters does Alice's brother have? To answer the question, DO NOT OUTPUT ANY TEXT EXCEPT following format that contains final answer: ### Answer:", "Alice has 4 brothers and she also has 1 sister. How many sisters does Alice's brother have?"], 'right_answer': ['2', '5', '2', '5', 

In [42]:
import openai

# Function to query the OpenAI model
client = openai.OpenAI(api_key=openai_api_key)

async def query_openai(prompt, model="o1-preview-2024-09-12"):
    try:
        response = await client.chat.completions.acreate(
            model=model,
            messages=[
                {"role": "user", "content": prompt},
            ],
        )
        return response.choices[0].message.content.strip(), response.usage
    except Exception as e:
        return str(e)

In [43]:
answer, usage = await query_openai("what is the capital of England")
print(answer)
print(usage)

ValueError: too many values to unpack (expected 2)

In [41]:
from tqdm.notebook import tqdm

# Validate OpenAI's model against the dataset
results = []

for item in tqdm(data):
    prompt = item["prompt"]
    expected = item["completion"]  # Adjust the key if necessary

    # Query the OpenAI model
    generated_answer, usage = await query_openai(prompt)

    # Record the results
    results.append({
        "prompt": prompt,
        "expected": expected,
        "generated": generated
    })


The capital of England is London.


In [ ]:
import pandas as pd



# Convert results to a DataFrame for analysis
results_df = pd.DataFrame(results)

# Calculate the percentage of correct responses
results_df["is_correct"] = results_df["expected"] == results_df["generated"]
accuracy = results_df["is_correct"].mean() * 100

# Save the results to a CSV file
results_df.to_csv("validation_results.csv", index=False)

# Display a preview of the results and accuracy
print(f"Accuracy: {accuracy:.2f}%")
results_df.head()
